In [0]:
# 5A: Load sampled data
df = spark.read.parquet(f"abfss://curated@{storage_account}.dfs.core.windows.net/criteo_uplift_1m")
print(f"✅ Loaded {df.count():,} rows")

# 5B: Feature engineering

# Optional: Rename f0–f11 to meaningful names like I1–I12
new_col_names = {
    'f0': 'I1', 'f1': 'I2', 'f2': 'I3', 'f3': 'I4',
    'f4': 'I5', 'f5': 'I6', 'f6': 'I7', 'f7': 'I8',
    'f8': 'I9', 'f9': 'I10', 'f10': 'I11', 'f11': 'I12'
}

for old_name, new_name in new_col_names.items():
    df = df.withColumnRenamed(old_name, new_name)

# Drop any rows with missing values
df = df.dropna()

# Cast all I-columns and labels to IntegerType
from pyspark.sql.types import IntegerType

for col in df.columns:
    df = df.withColumn(col, df[col].cast(IntegerType()))

# Show schema to confirm
df.printSchema()

# Save processed features to 'curated' container
df.write.mode("overwrite").parquet(f"abfss://curated@{storage_account}.dfs.core.windows.net/criteo-1m-features")
print("✅ Feature-engineered data written to curated/criteo-1m-features")

